In [1]:
%reset -f
#pip install tmdbsimple
import pandas as pd
import numpy as np
from numpy.random import shuffle
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB
import tmdbsimple as tmdb
tmdb.API_KEY = 'abcd9827ce3fb2298891370ff0cd0e58'

In [2]:
def takeInfos(id):
    atores = []
    crews = []
    keywords = []
    sTitles = []
    sPopularity = []
    sMedia = []

    movie = tmdb.Movies(id)

    response = movie.info()
    popularity = round(movie.popularity)
    media = round(movie.vote_average)
    title = movie.title
    for ator in movie.credits()['cast']: atores.append(ator['name'])
    for crew in movie.credits()['crew']: crews.append(crew['name'])
    for keyword in movie.keywords()['keywords']: keywords.append(keyword['name'])
    for similar in movie.similar_movies()['results']: 
        sTitles.append(similar['title'])
        sPopularity.append(round(similar["popularity"]))
        sMedia.append(round(similar["vote_average"]))
    
    return title,popularity,media,atores,crews,keywords,sTitles,sPopularity,sMedia

def NormalizeCat(X): #Não esta sendo usado
    global perda
    i = 0
    n = len(X)
    limit = 300
    if len(X)>limit:perda+=1
    while(len(X)<limit):
        if i == n: i = 0
        X.append(X[i])
        i+=1
    while(len(X)>limit):
        X.pop()
    return X
        

In [43]:
class Encoder:
    def __init__(self,lista):
        self.Codex = []
        self.atualIndex = 1
        self.fit(lista)
        
    def fit(self,lista):
        for item in lista:
            onList = False
            for code in self.Codex:
                if item == code[0]: onList = True
            if not onList:
                self.Codex.append([item,self.atualIndex])
                self.atualIndex+=1
    
    def transform(self,lista):
        resposta = []
        for item in lista:
            for code in self.Codex:
                if item == code[0]: resposta.append(code[1])
        return resposta
    
    def fit_transform(self,lista):
        self.fit(lista)
        return self.transform(lista)
    
    def inverse_transform(self,lista):
        resposta = []
        for item in lista:
            for code in self.Codex:
                if item == code[1]: resposta.append(code[0])
        return resposta
    
    def show(self):
        print(self.Codex)
    
    def Feed(self,id):
        title,popularity,media,atores,crews,keywords,sTitles,sPopularity,sMedia = takeInfos(id)
        self.fit([title])
        self.fit(sTitles)
        self.fit(atores)
        self.fit(crews)
        self.fit(keywords)

        Atores = self.transform(atores)
        Crews = self.transform(crews)
        Keywords = self.transform(keywords)
        X = self.fit_transform(["P"+str(popularity)])
        if len(Crews)>0:X.append(Crews[0])
        else: X.append(X[-1])
        for i in range(3):
            if len(Atores)>(i): X.append(Atores[i])
            else: X.append(X[-1])

            if len(Keywords)>(i): X.append(Keywords[i])
            else: X.append(X[-1])
        y = media
        return X,y

In [44]:
credits = pd.read_csv("tmdb_5000_credits.csv")
movies = pd.read_csv("tmdb_5000_movies.csv")

In [45]:
IDlist = list(movies["id"])
n = 3000
IDTrainList = IDlist[:n]
IDTestList = IDlist[n:]

In [46]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [47]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [48]:
print(X)

[[[94, 53, 33, 83, 34, 84, 35, 85]], [[145, 132, 115, 142, 116, 143, 117, 144]]]


In [63]:
#NÃO RODAR ISSO SE NÃO FOR MUUUITO NECESSARIO
Codex = Encoder(['M0',"M1","M2",'M3','M4','M5','M6','M7','M8','M9','M10'])
Naive = MultinomialNB()

In [64]:
n = 500
i = 0
X = []
y = []
shuffle(IDTrainList)
for idd in IDTrainList:
    X.append(Codex.Feed(idd)[0])
    y.append(Codex.Feed(idd)[1])
    i += 1
    print(str(i)+"/"+str(n)+" [{0}]".format(y[i-1]))
    if i == n:break
print("\nEnsinando...")
Naive.fit(X,y)

1/500 [7]
2/500 [8]
3/500 [6]
4/500 [7]
5/500 [4]
6/500 [5]
7/500 [7]
8/500 [6]
9/500 [4]
10/500 [4]
11/500 [5]
12/500 [5]
13/500 [5]
14/500 [8]
15/500 [6]
16/500 [7]
17/500 [5]
18/500 [6]
19/500 [7]
20/500 [7]
21/500 [6]
22/500 [7]
23/500 [5]
24/500 [6]
25/500 [7]
26/500 [7]
27/500 [8]
28/500 [6]
29/500 [5]
30/500 [7]
31/500 [7]
32/500 [7]
33/500 [6]
34/500 [7]
35/500 [7]
36/500 [7]
37/500 [7]
38/500 [5]
39/500 [7]
40/500 [6]
41/500 [7]
42/500 [6]
43/500 [7]
44/500 [7]
45/500 [6]
46/500 [6]
47/500 [6]
48/500 [7]
49/500 [6]
50/500 [6]
51/500 [5]
52/500 [6]
53/500 [7]
54/500 [6]
55/500 [6]
56/500 [7]
57/500 [7]
58/500 [7]
59/500 [6]
60/500 [5]
61/500 [6]
62/500 [7]
63/500 [6]
64/500 [7]
65/500 [6]
66/500 [7]
67/500 [6]
68/500 [5]
69/500 [6]
70/500 [4]
71/500 [6]
72/500 [6]
73/500 [6]
74/500 [7]
75/500 [3]
76/500 [7]
77/500 [4]
78/500 [7]
79/500 [6]
80/500 [6]
81/500 [7]
82/500 [6]
83/500 [5]
84/500 [6]
85/500 [6]
86/500 [6]
87/500 [8]
88/500 [7]
89/500 [6]
90/500 [8]
91/500 [6]
92/500 [

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [75]:
i = 0
n = 500
shuffle(IDTestList)
Matriz = [[0,0,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0,0,0,0]]
for idd in IDTestList: 
    X,y = Codex.Feed(idd)
    saida = Naive.predict([X])
    
    Matriz[saida[0]][y]+=1

    i += 1
    print(str(i)+"/"+str(n)+" [{0} , {1}]".format(saida[0],y))
    if i == n:break

1/500 [5 , 6]
2/500 [5 , 7]
3/500 [7 , 7]
4/500 [8 , 7]
5/500 [6 , 8]
6/500 [5 , 4]
7/500 [7 , 4]
8/500 [3 , 6]
9/500 [8 , 5]
10/500 [3 , 5]
11/500 [5 , 6]
12/500 [5 , 7]
13/500 [5 , 8]
14/500 [6 , 5]
15/500 [5 , 7]
16/500 [3 , 6]
17/500 [5 , 6]
18/500 [7 , 6]
19/500 [8 , 0]
20/500 [7 , 6]
21/500 [3 , 4]
22/500 [7 , 7]
23/500 [7 , 7]
24/500 [5 , 5]
25/500 [5 , 6]
26/500 [3 , 8]
27/500 [7 , 6]
28/500 [5 , 5]
29/500 [5 , 7]
30/500 [8 , 8]
31/500 [5 , 7]
32/500 [4 , 5]
33/500 [7 , 7]
34/500 [5 , 5]
35/500 [4 , 7]
36/500 [6 , 6]
37/500 [3 , 7]
38/500 [6 , 7]
39/500 [4 , 6]
40/500 [5 , 6]
41/500 [8 , 3]
42/500 [8 , 4]
43/500 [5 , 6]
44/500 [5 , 7]
45/500 [5 , 5]
46/500 [3 , 7]
47/500 [3 , 7]
48/500 [7 , 6]
49/500 [4 , 6]
50/500 [5 , 7]
51/500 [7 , 7]
52/500 [5 , 8]
53/500 [4 , 6]
54/500 [5 , 5]
55/500 [5 , 7]
56/500 [5 , 6]
57/500 [5 , 4]
58/500 [6 , 6]
59/500 [8 , 7]
60/500 [3 , 7]
61/500 [3 , 7]
62/500 [5 , 6]
63/500 [5 , 5]
64/500 [3 , 6]
65/500 [5 , 6]
66/500 [3 , 7]
67/500 [4 , 7]
68/5

In [ ]:
search = tmdb.Search()
response = search.movie(query='The Man Who Invented')
for s in search.results: print(s['title'], s['id'], s['release_date'], s['popularity'], round(s['vote_average']))

In [ ]:
for linha in Matriz:
    print(linha)

In [ ]:
AcertoB = 0
AcertoC = 0
Erro = 0
for i in range(len(Matriz[0])):
    AcertoB += Matriz[i][i]
    if i !=0:
        AcertoC+= Matriz[i-1][i]
        AcertoC+= Matriz[i][i-1]
    Erro+= sum(Matriz[i])
Erro -= AcertoC+AcertoB

print("AcertoB: {}".format(AcertoB))
print("AcertoC: {}".format(AcertoC))
print("Erro: {}".format(Erro))